In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [ ]:
url = "https://lazyprogrammer.me/course_files/nlp/tmdb_5000_movies.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
x = df.iloc[0]
x

In [ ]:
x['genres']

In [ ]:
x['keywords']

In [ ]:
j = json.loads(x['genres'])
j

In [ ]:
' '.join(''.join(jj['name'].split())for jj in j)

In [ ]:
#convert the relevant data for each movie into a single string
#to be ingested by TfidfVectorizer
def genres_and_keywords_to_string(row):
    genres = json.loads(row['genres'])
    genres = ' '.join(''.join(j['name'].split())for j in genres)
    keywords = json.loads(row['keywords'])
    keywords = ' '.join(''.join(j['name'].split())for j in keywords)
    return "{} {}".format(genres, keywords)

In [ ]:
#create a new representation of each movie
df['string'] = df.apply(genres_and_keywords_to_string, axis=1)

In [ ]:
#create a tf-idf vectorizer object
tfidf = TfidfVectorizer(max_features = 2000)

In [ ]:
#create a data matrix from the overviews
X = tfidf.fit_transform(df['string'])

In [ ]:
X

In [ ]:
#generate a mapping from moive title 
movie2idx = pd.Series(df.index, index=df['title'])
movie2idx

In [ ]:
 idx = movie2idx['Avatar']
idx

In [ ]:
query = X[idx]
query.toarray()

In [ ]:
#compute similarity between query and every vector in X
scores = cosine_similarity(query, X)
scores

In [ ]:
#currently the array is 1 * n, make it just a 1-D array
scores = scores.flatten()
plt.plot(scores)

In [ ]:
(-scores).argsort()

In [ ]:
plt.plot(scores[(-scores).argsort()])

In [ ]:
#get top 5 matches
recommended_idx = (-scores).argsort()[1:6]

In [ ]:
#convert indices back to titles
df['title'].iloc[recommended_idx]

In [ ]:
#create a function that generates recommendations
def recommend(title):
    #get the row in the dataframe for this movie
    idx = movie2idx[title]
    if type(idx) == pd.Series:
        idx = idx.iloc[0]
    #calculate the pairwise similarities for this movie
    query = X[idx]
    scores = cosine_similarity(query, X)
    scores = scores.flatten()
    recommended_idx = (-scores).argsort()[1:6]

    #return the titles of recommendations
    return df['title'].iloc[recommended_idx]



In [ ]:
print(recommend('Mortal Kombat'))

In [ ]:
print(recommend('Avatar'))